In [1]:
import numpy as np
from astropy import units as u
from astropy import constants as const
from astropy.io import ascii
from scipy import interpolate
from bit_tools.mission_tools import Mission
from mag_converter import *

/home/veome/anaconda3/lib/python3.8/site-packages/pyds9.py:94: UserWarning: Can't locate DS9 executable. Please add the DS9 directory to your PATH and try again.
  warnings.warn(ds9_warning)


In [2]:
magcut = 26.0
DIRECTORY = '/mnt/44868D0F868D02A2/Users/veome/Google Drive/Academics/Senior Thesis/Code/Catalog/'

In [3]:
cosmos = np.genfromtxt(DIRECTORY+'acs_cosmos1361.gcat_ascii', usecols=range(0,11), invalid_raise=False)

<ipython-input-3-0a010cf2ec54>:1: ConversionWarning: Some errors were detected !
    Line #2617 (got 10 columns instead of 11)
  cosmos = np.genfromtxt(DIRECTORY+'acs_cosmos1361.gcat_ascii', usecols=range(0,11), invalid_raise=False)


In [4]:
# Get HST F814W Band from bit_tools/bit_tools/data/missions/hst/bandpass/
hst = Mission("hst")
band = hst.get_bandpass("f814w")
f814w_band_response = band.r_lam
f814w_pivot = band.lam_pivot * (u.nm.to(u.AA))


# Change units from nm to Angstrom
# x = band.get_wavelengths() 
x = np.arange(300, 1200, 1)
f814w = interpolate.interp1d(x*10, f814w_band_response(x), fill_value="extrapolate")

In [5]:
# Get SuperBIT Shape Band from bit_tools/bit_tools/data/missions/superbit/bandpass/
# bit = Mission("superbit19")
# r_shape = bit.get_bandpass("lum").r_lam

bit = Mission("superbit")
band = bit.get_bandpass("shape")
r_shape = band.r_lam
shape_pivot = band.lam_pivot * (u.nm.to(u.AA))



# Change units from nm to Angstrom
# x = bit.get_bandpass("shape").get_wavelengths()
x = np.arange(300, 1200, 1)
sbit_shape = interpolate.interp1d(x*10, r_shape(x), fill_value="extrapolate")

In [6]:
# Elliptical Template
elliptical_template = np.genfromtxt('galaxy_spectra_kinney96/elliptical_template.ascii', usecols=range(2), invalid_raise=False)

In [7]:
# Starburst Template
starburst_all = [[], [], [], [], [], []]
for i in range(1, 7):
    starburst_all[i-1] = np.genfromtxt('galaxy_spectra_kinney96/starb%i_template.ascii' %(i), usecols=range(2), invalid_raise=False)

starburst_template = np.mean( np.array([starburst_all[0], starburst_all[1], starburst_all[2], starburst_all[3], starburst_all[4], starburst_all[5]]), axis=0 )

In [8]:
# Initialization
cat = cosmos[cosmos[:,2] > 0] # redshift cut
cat = cat[cat[:,5] <= magcut] # mag cut
length = cat.shape[0]
width = cat.shape[1]

print("Original Catalog Size: %i"%cosmos.shape[0])
print("New Catalog Size: %i"%length)

Original Catalog Size: 697594
New Catalog Size: 389181


In [9]:
# Mag conversion
z = cat[:,2]
mag_obs = cat[:,5]
super_e_mag = np.zeros(length)
super_starb_mag = np.zeros(length)

for i in range(length):
    if (i%(10000) == 0):
        percentage = int((i/length)*100)
        print("Object {} of {} ({}%)".format(i, length, percentage))
        
    # ST Mag Conversion    
#     super_e_mag[i], super_starb_mag[i] = stmag_conversion(mag_obs[i], z[i], f814w, sbit_shape, elliptical_template, starburst_template)  
    
    # AB Mag Conversion    
    super_e_mag[i], super_starb_mag[i] = abmag_conversion(mag_obs[i], z[i], f814w, sbit_shape, elliptical_template, starburst_template, src_pivot=f814w_pivot, target_pivot=shape_pivot)    
    
print("Conversion complete!")

Object 0 of 389181 (0%)
Object 10000 of 389181 (2%)
Object 20000 of 389181 (5%)
Object 30000 of 389181 (7%)
Object 40000 of 389181 (10%)
Object 50000 of 389181 (12%)
Object 60000 of 389181 (15%)
Object 70000 of 389181 (17%)
Object 80000 of 389181 (20%)
Object 90000 of 389181 (23%)
Object 100000 of 389181 (25%)
Object 110000 of 389181 (28%)
Object 120000 of 389181 (30%)
Object 130000 of 389181 (33%)
Object 140000 of 389181 (35%)
Object 150000 of 389181 (38%)
Object 160000 of 389181 (41%)
Object 170000 of 389181 (43%)
Object 180000 of 389181 (46%)
Object 190000 of 389181 (48%)
Object 200000 of 389181 (51%)
Object 210000 of 389181 (53%)
Object 220000 of 389181 (56%)
Object 230000 of 389181 (59%)
Object 240000 of 389181 (61%)
Object 250000 of 389181 (64%)
Object 260000 of 389181 (66%)
Object 270000 of 389181 (69%)
Object 280000 of 389181 (71%)
Object 290000 of 389181 (74%)
Object 300000 of 389181 (77%)
Object 310000 of 389181 (79%)
Object 320000 of 389181 (82%)
Object 330000 of 389181 (84%

In [10]:
all_data = np.zeros((length,width+2))
all_data[:,0:width] = cat
all_data[:,width] =  np.around(super_e_mag, decimals=5)
all_data[:,width+1] = np.around(super_starb_mag, decimals=5)

In [11]:
headers = np.array(['# RA [deg]', 'Dec [deg]', 'Redshift', 'gamma1', 'gamma2', 'Magnitude', 'Weight', 'Size', 'UncorSize', 'FWHM', 'SNR_HST', 'SuperBIT E Mag', 'SuperBIT Starburst Mag'])

In [12]:
fname = 'superbit_cosmos1361_shape_abmag26.gcat_ascii'
ascii.write(all_data, fname, delimiter=' ', format='fixed_width', names=headers, overwrite=True)
print("Catalog written to {}".format(fname))

Catalog written to superbit_cosmos1361_shape_abmag26.gcat_ascii


In [13]:
np.amax(super_e_mag)

27.674499560175427